#### Day-3

Agenda: 
1. PromptTemplates
2. Chains
3. Output Parser

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

## Chat Completion Model

In [4]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
from langchain_groq import ChatGroq
llamaModel = ChatGroq(
    model="llama3-70b-8192"
)

## Prompts and Prompt Templates

In [10]:
# This is for Completion Model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}"
)

llmModelPrompt = prompt_template.format(
    adjective="curious",
    topic="Tesla"
)
response = llamaModel.invoke(llmModelPrompt)
response.content

'Here\'s a curious story about Nikola Tesla:\n\n**The Mysterious Case of the Electric Oscillator**\n\nIn the early 1890s, Nikola Tesla was working on a top-secret project in his laboratory in New York City. He had been experimenting with a new device he called the "electric oscillator," which was capable of producing intense, high-voltage electrical discharges.\n\nOne day, Tesla decided to test the oscillator\'s power by attaching it to a metal rod and burying it in the ground outside his laboratory. He then turned on the device and stood back to observe its effects.\n\nTo his amazement, the oscillator began to create a strange, pulsating hum that seemed to vibrate the very air around it. The hum grew louder and louder, until it was almost deafening. Suddenly, the metal rod began to glow with a brilliant, electric blue light, and the air around it started to shimmer and distort.\n\nTesla was thrilled with the results, but he didn\'t realize that his experiment was about to take a bizar

In [12]:
# This is for Chat Completion Model
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}"),
        ("human", "Hello Mr. {profession}, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),

    ]
)

messages= chat_template.format_messages(
    profession="Historian",
    topic="Nikola Tesla",
    user_input="Tell me the struggle of Nikola Tesla?"
)

response = llamaModel.invoke(messages)
response.content

'The struggles of Nikola Tesla! Where to begin? (laughs) Well, let me tell you, Tesla\'s life was a rollercoaster of triumphs and tribulations.\n\nFirstly, Tesla\'s early life was marked by struggle. Born in 1856 in Smiljan, Croatia, he was the fourth of five children. His father, Rev. Milutin Tesla, was a Serbian Orthodox priest and a writer, but he struggled to provide for his family. Tesla\'s mother, Đuka, managed the family\'s farm, but the family lived in poverty. Tesla\'s early education was marked by a passion for science and technology, but his parents couldn\'t afford to send him to a good university.\n\nTesla\'s determination and hard work eventually earned him a scholarship to the University of Prague, where he studied electrical engineering. However, his time in Prague was cut short due to his mother\'s illness and eventual death. Tesla returned to Croatia, but he soon fell ill with cholera and was bedridden for nine months.\n\nAfter recovering, Tesla moved to Budapest, whe

## Old Prompt Template

In [22]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("You are an expert on the kennedy family")
        ),
        HumanMessagePromptTemplate.from_template(
            "{user_input}"
        ),

    ]
)

messages = chat_template.format_messages(
    user_input = "Name the children and grandchildren of Jospeh P. kennedy?"
)

response = llamaModel.invoke(messages)
response.content

'Joseph P. Kennedy Sr. had nine children with his wife Rose Fitzgerald Kennedy:\n\n1. Joseph P. Kennedy Jr. (1915-1944)\n\t* No children\n2. John F. Kennedy (1917-1963)\n\t* Arabella Kennedy (1956-1956)\n\t* Caroline Bouvier Kennedy (b. 1957)\n\t* John Fitzgerald Kennedy Jr. (1960-1999)\n\t* Patrick Bouvier Kennedy (1963-1963)\n3. Rosemary Kennedy (1918-2005)\n\t* No children\n4. Kathleen Kennedy (1920-1948)\n\t* No children\n5. Eunice Kennedy (1921-2009)\n\t* Maria Shriver (b. 1955)\n\t* Timothy Perry Shriver (b. 1959)\n\t* Mark Kennedy Shriver (b. 1964)\n\t* Anthony Paul Kennedy Shriver (b. 1965)\n6. Patricia Kennedy (1924-2006)\n\t* Christopher George Kennedy Lawford (b. 1955)\n\t* Sydney Maleia Kennedy Lawford (b. 1956)\n\t* Victoria Francis Lawford (b. 1958)\n\t* Robin Elizabeth Lawford (b. 1961)\n7. Robert F. Kennedy (1925-1968)\n\t* Kathleen Kennedy Townsend (b. 1951)\n\t* Joseph Patrick Kennedy II (b. 1952)\n\t* Robert Francis Kennedy Jr. (b. 1954)\n\t* David Anthony Kennedy (1

## Few Shot Prompting

In [26]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {"input": "hi", "output": "hola!"},
    {"input": "bye!", "output": "adios"},    
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish Translator"),
        few_shot_prompt,
        ("human", "{input}")

    ]
)

## Chains

In [30]:
# Langchain Expression Language syntax
chain = final_prompt | llamaModel

In [31]:
response = chain.invoke({"input": "How are you?"})
response.content

'Estoy bien, ¿y tú?'

## Output Parsers

In [39]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

prompt = PromptTemplate.from_template(
    "Return a JSON object with an answer key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
print(json_parser.get_format_instructions())
chain = prompt | llamaModel | json_parser

response = chain.invoke({"question": "Which is the biggest country in the world?"})
response

Return a JSON object.


{'question': 'Which is the biggest country in the world?',
 'answer': 'Russia',
 'explanation': 'Russia is the largest country in the world by land area, covering approximately 17.1 million square kilometers (6.6 million square miles).'}

Optionally, you can use Pydantic to define a custom output format

In [44]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define a pydantic object with the desired output format
class Dummy(BaseModel):
    question: str = Field(description="Some dummy question to ask")
    answer: str = Field(description="answer to the dummy question")

In [45]:
# Define parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Dummy)
prompt = PromptTemplate(
    template="Answer the user query. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}

)

chain = prompt | llamaModel | parser

chain.invoke({"query": "Tell me something about anything."})

{'question': 'What is the meaning of life?',
 'answer': 'A question that has puzzled philosophers and scholars for centuries, and still remains a mystery to this day.'}